In [1]:
import numpy as np

from sklearn import metrics
from sklearn import preprocessing 

from sklearn.linear_model import Perceptron

In [21]:
def convert_numerical_cols(data, numeric_cols):    
    for col in numeric_cols:
        median = np.median(data[:, col].astype(float))
        for d in data: 
            if float(d[col]) <= median:
                d[col] = 0
            else:
                d[col] = 1
    return data

def convert_categorical_cols(data, categorical_cols):
    n_row, n_col = data.shape
    
    for col in categorical_cols:
        s = data[:,col:col+1]
        _cats = np.unique(s).reshape(-1, 1)
        ohe = preprocessing.OneHotEncoder()
        ohe.fit(_cats)
        d = ohe.transform(_cats).todense()
        for i, _cat in enumerate(_cats):
            np.putmask(s, s==_cat, d[i])
            # s[s==_cat] = d[i]
        
        data[:,col:col+1] = s

    return data

def process(file):
    data = np.genfromtxt(file, delimiter=',', dtype='str')

    data = data[1:data.shape[0],]
    n_row, n_col = data.shape

    return data[:,0:n_col-1], data[:,n_col-1]

def process_test(file):
    data = np.genfromtxt(file, delimiter=',', dtype='str')

    data = data[1:data.shape[0],]
    n_row, n_col = data.shape

    return data[:,0:1], data[:,1:n_col]

In [3]:
numeric_cols = [0, 2, 4, 10, 11, 12]
categorical_cols = [1, 3, 5, 6, 7, 8, 9, 13]

In [22]:
## read data
X, Y = process('./data/train_final.csv')

X = convert_numerical_cols(X, numeric_cols)
X = convert_categorical_cols(X, categorical_cols)


# create decision tree
clf = Perceptron(tol=1e-3, random_state=0).fit(X, Y)

TypeError: Cannot cast array data from dtype('float64') to dtype('<U26') according to the rule 'safe'

In [5]:
test_numeric_cols = [0, 2, 4, 10, 11, 12]
test_categorical_cols = [1, 3, 5, 6, 7, 8, 9, 13]
_ids, TEST_X = process_test('./data/test_final.csv')

print(TEST_X.shape)
TEST_X = convert_numerical_cols(TEST_X, test_numeric_cols)
TEST_X = convert_categorical_cols(TEST_X, test_categorical_cols)

(23842, 14)


In [6]:
# probability of each class
results = clf.predict_proba(TEST_X)

AttributeError: 'Perceptron' object has no attribute 'predict_proba'